In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [4]:
design.get_chip_z('Q_chip')

'12um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='12um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2-0.2
    resonator_length_2.append(a)
print(resonator_length_2)


[7.253254683870284, 7.024788339841421, 6.80834654020913]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            pad_gap='20um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+75}um",
                                              pad_width="110um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 06:53AM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:53AM [load_ansys_project]: 	Opened Ansys App
INFO 06:53AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:53AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 06:53AM [connect_design]: No active design found (or error getting active design).
INFO 06:53AM [connect]: 	 Connected to project "Project20". No design detected


True

In [15]:
hfss.activate_ansys_design("flip12um_V1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

06:53AM 18s WARNING [activate_ansys_design]: The design_name=flip12um_V1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 06:53AM [connect_design]: 	Opened active design
	Design:    flip12um_V1 [Solution type: Eigenmode]
WARNING 06:53AM [connect_setup]: 	No design setup detected.
WARNING 06:53AM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:53AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


06:54AM 24s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.metal.py' not found in 'All Files (*)'.

06:54AM 24s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.metal.py' not found in 'All Files (*)'.



In [16]:
hfss.close()

True

# LOM Analysis

In [17]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [18]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.34',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.34',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [20]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 09:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpiig5kcm7.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5340000000, Maxwell, 1, False
INFO 09:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp_jx6x07u.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5340000000, Maxwell, 1, False
INFO 09:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpol4v0d68.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5340000000, Maxwell, 2, False
INFO 09:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpcm79k0_5.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5340000000, Maxwell, 3, False
INFO 09:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpxt28hozt.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5340000000, Maxwell, 4, False
INFO 09:06

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,404.440,-89.4610,-56.237,-198.3100
pad_bot_Q0,-89.461,127.6200,-36.223,-0.8494
pad_top_Q0,-56.237,-36.2230,127.590,-34.0580
quad_coupler_c0,-198.310,-0.8494,-34.058,234.4200


In [21]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.00
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.011393 [GHz]
EC 232.328386 [MHz]
EJ 14.854145 [GHz]
alpha -261.659727 [MHz]
dispersion 0.927780 [KHz]
Lq 10.995564 [nH]
Cq 83.374350 [fF]
T1 29.443471 [us]

**Coupling Properties**

tCqbus1 -16.605788 [fF]
gbus1_in_MHz -209.421154 [MHz]
χ_bus1 -5.291532 [MHz]
1/T1bus1 5405.440974 [Hz]
T1bus1 29.443471 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.45524,277.994685,14.854145,-317.483322,6.085123,[-17.528592738611042],[-0.06910887223576853],0.069109,17.528593
2,5.219142,253.131491,14.854145,-286.924756,2.329862,[-288.2628178529002],[-13.276199247251483],13.276199,288.262818
3,5.147176,245.812161,14.854145,-278.004046,1.708107,[-283.4945954260936],[-11.628422742087766],11.628423,283.494595
4,5.153406,246.44099,14.854145,-278.769112,1.755239,[-272.2621160297716],[-10.816763521575563],10.816764,272.262116
5,5.117524,242.83126,14.854145,-274.380761,1.499155,[-241.29577259690348],[-8.091868877602495],8.091869,241.295773
6,5.081003,239.187885,14.854145,-269.959941,1.273887,[-224.82396163188716],[-6.688113797164824],6.688114,224.823962
7,5.053399,236.454411,14.854145,-266.648748,1.124584,[-216.9672811838336],[-6.003783303643699],6.003783,216.967281
8,5.03657,234.796508,14.854145,-264.642773,1.041573,[-213.37559905702463],[-5.678586550659205],5.678587,213.375599
9,5.02644,233.801765,14.854145,-263.440028,0.994343,[-211.90849172047288],[-5.526349804602084],5.526350,211.908492
10,5.011393,232.328386,14.854145,-261.659727,0.92778,[-209.4211544040327],[-5.291531804572647],5.291532,209.421154


In [22]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
